# Australian Geoscience Datacube API
This notebook describes connecting to the datacube and doing a basic query

In [36]:
import datacube.api
from pprint import pprint

By default, the API will use the configured database connection found in the config file.

Details on setting up the config file and database and be found here:
http://agdc-v2.readthedocs.org/en/develop/db_setup.html

In [37]:
dc = datacube.api.API()

## Summary functions
* __`list_fields()`__ - lists all fields that can be used for searching
* __`list_field_values(field)`__ - lists all the values of the field found in the database

Find out what fields we can search:

In [29]:
dc.list_fields()

[u'product',
 u'lat',
 u'sat_path',
 u'platform',
 u'lon',
 u'orbit',
 'collection',
 u'instrument',
 u'sat_row',
 u'time',
 u'gsi',
 'id']

The `product` and `platform` fields looks interesting. Find out more about them:

In [30]:
dc.list_field_values('product')

[u'LEDAPS']

In [31]:
dc.list_field_values('platform')

[u'LANDSAT_5']

## Query and Access functions
There are several API calls the describe and provide data in different ways:

* __`get_descriptor()`__ - provides a descripton of the data for a given query
* __`get_data()`__ - provides the data as `xarray.DataArray`s for each variable.  This is usually called based on information returned by the `get_descriptor` call.
* __`get_data_array()`__ - returns an `xarray.DataArray` n-dimensional object, with the variables stack along the dimension labelled `variables`.
* __`get_dataset()`__ - return an `xarray.Dataset` object, containing an `xarray.DataArray` for each variable.

###  get_descriptor
We can make a query and find out about the data:

The query is a nested dict of variables of terms.

In [38]:
query = {
    'product': 'LEDAPS',
    'platform': 'LANDSAT_5',
}
descriptor = dc.get_descriptor(query, include_storage_units=False)
pprint(descriptor)

{u'ls5_ledaps_albers': {'dimensions': [u'time', u'y', u'x'],
                        'irregular_indices': {u'time': array(['2011-10-17T11:15:48.000000000+1100'], dtype='datetime64[ns]')},
                        'result_max': (numpy.datetime64('2011-10-17T11:15:48.000000000+1100'),
                                       -3800012.5,
                                       899987.5),
                        'result_min': (numpy.datetime64('2011-10-17T11:15:48.000000000+1100'),
                                       -4099987.5,
                                       600012.5),
                        'result_shape': (1, 12000, 12000),
                        'variables': {u'band1': {'datatype_name': dtype('int16'),
                                                 'nodata_value': -9999},
                                      u'band2': {'datatype_name': dtype('int16'),
                                                 'nodata_value': -9999},
                                      u'band3': {'d

The query can be restricted to provide information on particular range along a dimension.

For spatial queries, the dimension names should be used.  The default projection for the range query values is in WGS84, although

In [42]:
query = {
    'product': 'LEDAPS',
    'platform': 'LANDSAT_5',
    'dimensions': {
        'x' : {
            'range': (600012.5, 899987.5),
            'crs': 'EPSG:3577',
        },
        'y' : {
            'range': (-4099987.5, -3800012.5),
            'crs': 'EPSG:3577',
        },
        'time': {
            'range': ((2011, 10, 17), (2011, 10, 18)),
        }
    }
}
pprint(dc.get_descriptor(query, include_storage_units=False))

{u'ls5_ledaps_albers': {'dimensions': [u'time', u'y', u'x'],
                        'irregular_indices': {u'time': array(['2011-10-17T11:15:48.000000000+1100'], dtype='datetime64[ns]')},
                        'result_max': (numpy.datetime64('2011-10-17T11:15:48.000000000+1100'),
                                       -3800012.5,
                                       899987.5),
                        'result_min': (numpy.datetime64('2011-10-17T11:15:48.000000000+1100'),
                                       -4099987.5,
                                       600012.5),
                        'result_shape': (1, 12000, 12000),
                        'variables': {u'band1': {'datatype_name': dtype('int16'),
                                                 'nodata_value': -9999},
                                      u'band2': {'datatype_name': dtype('int16'),
                                                 'nodata_value': -9999},
                                      u'band3': {'d

A coordinate reference sytsem can be provided for the spatial dimensions, either as a EPSG code or a WKT description:

In [ ]:
query = {
    'product': 'LEDAPS',
    'platform': 'LANDSAT_5',
    'dimensions': {
        'x' : {
            'range': (600012.5, 899987.5),
            'crs': 'EPSG:3577',
        },
        'y' : {
            'range': (-4099987.5, -3800012.5),
            'crs': 'EPSG:3577',
        },
        'time': {
            'range': ((2011, 10, 17), (2011, 10, 18)),
        }
    }
}

### get_data
This retrieves the data, usually as a subset, based on the information provided by the `get_descriptor` call.

The query is in a similar form to the `get_descriptor` call, with the addition of a `variables` parameter.  If not specified, all variables are returned.
The query also accepts an `array_range` parameter on a dimension that provides a subset based on array indicies, rather than labelled coordinates.

In [22]:
query = {
    'product': 'LEDAPS',
    'platform': 'LANDSAT_5',
    'variables': ['band1', 'band2'],
    'dimensions': {
        'x' : {
            'range': (600012.5, 899987.5),
            'crs': 'EPSG:3577',
            'array_range': (0, 1),
        },
        'y' : {
            'range': (-4099987.5, -3800012.5),
            'crs': 'EPSG:3577',
            'array_range': (0, 1),
        },
        'time': {
            'range': ((2011, 10, 17), (2011, 10, 18)),
        }
    }
}
data = dc.get_data(query)
data.keys()

['dimensions',
 'arrays',
 'element_sizes',
 'indices',
 'coordinate_reference_systems',
 'size']

### get_data_array
This is a convinence function that wraps the `get_data` function, returning only the data, stacked in a single `xarray.DataArray`.

The variables are stacked along the `variable` dimension.

In [24]:
nbar = dc.get_data_array(product='LEDAPS', platform='LANDSAT_5')
nbar

<xarray.DataArray u'ls5_ledaps_albers' (variable: 16, time: 1, y: 12000, x: 12000)>
dask.array<concate..., shape=(16, 1, 12000, 12000), dtype=float64, chunksize=(1, 1, 4000, 4000)>
Coordinates:
  * time      (time) datetime64[ns] 2011-10-17T00:15:48
  * y         (y) float64 -3.8e+06 -3.8e+06 -3.8e+06 -3.8e+06 -3.8e+06 ...
  * x         (x) float64 6e+05 6e+05 6.001e+05 6.001e+05 6.001e+05 ...
  * variable  (variable) <U20 u'band1' u'band2' u'band3' u'band4' u'band5' ...

### get_dataset
This is a convinience fuction similar to `get_data_array`, returning the data of the query as a `xarray.Dataset` object.

In [35]:
dc.get_dataset(product='LEDAPS', platform='LANDSAT_5', set_nan=False)

<xarray.Dataset>
Dimensions:               (time: 1, x: 12000, y: 12000)
Coordinates:
  * time                  (time) datetime64[ns] 2011-10-17T00:15:48
  * y                     (y) float64 -3.8e+06 -3.8e+06 -3.8e+06 -3.8e+06 ...
  * x                     (x) float64 6e+05 6e+05 6.001e+05 6.001e+05 ...
Data variables:
    sr_cloud_shadow_qa    (time, y, x) int16 255 255 255 255 255 255 255 255 ...
    sr_snow_qa            (time, y, x) int16 255 255 255 255 255 255 255 255 ...
    band2                 (time, y, x) int16 -9999 -9999 -9999 -9999 -9999 ...
    band3                 (time, y, x) int16 -9999 -9999 -9999 -9999 -9999 ...
    band1                 (time, y, x) int16 -9999 -9999 -9999 -9999 -9999 ...
    band7                 (time, y, x) int16 -9999 -9999 -9999 -9999 -9999 ...
    band4                 (time, y, x) int16 -9999 -9999 -9999 -9999 -9999 ...
    band5                 (time, y, x) int16 -9999 -9999 -9999 -9999 -9999 ...
    sr_fill_qa            (time, y, x) int